In [ ]:
# Load the data and libraries
import pandas as pd
import random
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

adult = pd.read_csv('adult_with_pii.csv')

In [ ]:
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

In [ ]:
def people_over_30_dp(epsilon):
    true_answer = len(adult[adult['Age'] > 30])
    return laplace_mech(true_answer, 1, epsilon)

people_over_30_dp(epsilon = 1)

In this case, the sensitivity is equal to 1 because adding or removing a single person from the dataset can change the count of people over 30 by at most 1.

In [ ]:
def dp_crosstab_education_sex(epsilon):
    ct  = pd.crosstab(adult['Education'], adult['Sex'])
    sensitivity = 1
    noisy_ct = ct.applymap(lambda x: laplace_mech(x, sensitivity, epsilon))

    return noisy_ct

dp_crosstab_education_sex(1.0)

For a histogram with any number of dimensions (here 2), parallel composition holds because adding or removing an individual in the data changes exactly one count in the histogram. 

In terms of privacy cost, the number of columns doesn't matter because the sensitivity remains the same regardless of the number of dimensions. Each change in the dataset affects only one count in the histogram.

However, accuracy is crucial. The more columns you group by, the smaller the count will be in each resulting cell of the histogram. While the noise level remains constant, the signal-to-noise ratio decreases as the counts become smaller, potentially affecting the accuracy of the query results.

In [1]:
adult['Marital Status'].value_counts()

options = ['Never-married', 'Married-civ-spouse', 'Divorced',
           'Married-spouse-absent', 'Separated', 'Married-AF-spouse',
           'Widowed']


def score(option):
    count = len(adult[adult['Marital Status'] == option])
    return count

score('Never-married')

NameError: name 'adult' is not defined

Sensitivity is 1, using a counting query

In [ ]:
def most_common_marital_status(R, score, sensitivity, epsilon):
    #  Calculate scores for each option
    scores = [score(r) for r in R]

    # Add Laplace noise to each score
    noisy_scores = [laplace_mech(s, sensitivity, epsilon) for s in scores]

    # Select the option with the highest noisy score
    max_idx = np.argmax(noisy_scores)
    selected_option = R[max_idx]

    return selected_option
    
most_common_marital_status(options, score, 1, 1)

- When considering 7 options for R, the Laplace mechanism is employed 7 times, with a sensitivity of 1 for the scoring function. According to sequential composition, the cumulative privacy cost becomes 7 times the epsilon value. Through post-processing, the ultimate privacy cost remains at 7 times epsilon. Since reporting the noisy maximum is akin to using the exponential mechanism, the final deduction is that the total privacy cost is reduced to just epsilon. Answer: epsilon, so here 1.
- Moreover the algorithm defined above satisfies e-differential privacyt, regardless of set size.  because it releases only the identity of the element with the largest noisy count. The proof can be found in Dwork and Roth

In [ ]:
def dp_sum_capgain(epsilon):        
    upper_bound = adult['Capital Gain'].max() # The upper bound of our data
    clipped_sum = adult['Capital Gain'].clip(lower = 0, upper = upper_bound).sum()
    noisy_sum = laplace_mech(clipped_sum, upper_bound + 1, epsilon)
    
    return noisy_sum

dp_sum_capgain(1.0)

- The clipping parameter I chose for the dp_sum_capital function was the max value for the dataset, 99,999. I chose this value because there is no person exhibting more than a value of 99,999 for the capital gain column and thus it is safe to assume 100% of the database is included within the clipping.
  
- The sensitivity of the query used to compute dp_sum_capital is 100,000. This is because a summation query is unbounded in nature, meaning if a new row were added to the dataset the summation query would then be offset by the amount of the added row. But since our data ranges from 0 to 99,999 it is safe to safe that 100,00 will account for any value in our query that should be added into our dataset, so we can assume a sensitivity of the difference of the lower and upper bounds (with a little padding).

- My definition of dp_sum_capital has a total privacy cost of epsilon. This is because we are only creating one noisy sum here, so the amount of privacy we are using is exactly what is passed in as an argument.

In [ ]:
def age_cdf():
    a = adult['Age']
    return [len(a[a < i]) for i in range(100)]

plt.plot(age_cdf());
print('Length of CDF vector:', len(age_cdf()))

- The sensitivity of each individual count (element-wise sensitivity) in the vector is 1, as adding or removing a single individual can change the count by at most 1.

- The squared sensitivity of each individual count is 1 * 1 = 1.

- Since there are 100 counts in the vector, the L2 global sensitivity is the square root of the sum of the squares of the element-wise sensitivities, which is 100.

- The L1 sensitivity is the sum of the individual sensitivities, i.e., 100.

In [ ]:
def gaussian_mech_RDP(val, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return val + np.random.normal(loc=0, scale=sigma)

def gaussian_mech_RDP_vec(vec, sens, alpha, epsilon):
    return [gaussian_mech_RDP(x, sens, alpha, epsilon) for x in vec]

In [ ]:
def calculate_cdf(df, epsilon=1.0):
    true_cdf_vals = age_cdf()
    
    alpha = 5
    epsilon_bar = 0.001
    
    L2_sens = np.sqrt(len(df))
    
    noisy_cdf = gaussian_mech_RDP_vec(true_cdf_vals, L2_sens, alpha, epsilon)
    
    return noisy_cdf

# Calculate CDF with differential privacy
cdf_vals = calculate_cdf(adult)
# Plot CDF
plt.plot(cdf_vals)


- What is the *total privacy cost* in RDP of your solution above?

Utilize the L2 sensitivity to calculate the cumulative distribution function (CDF) vector. Each query within this vector is a counting query with a sensitivity of 1. Consequently, the overall L2 sensitivity amounts to the square root of len(data). Apply the Gaussian mechanism once, incurring a total privacy cost of (alpha, epsilon)-RDP, so alpha = 5.

- What is the *total privacy cost* in $(\epsilon, \delta)$-differential privacy of your solution above, for $\delta = 10^{-5}$? Implement the function `convert_RDP_ED` to convert the RDP parameters to $(\epsilon, \delta)$-DP parameters.

In [ ]:
def convert_RDP_ED(alpha, epsilon_bar, delta):
    epsilon = epsilon_bar + (np.log(1/delta) / (alpha -1))
    return epsilon

convert_RDP_ED(5, .0001, 1e-5)

The total privacy cost is (2.878, 1e-5)-differential privacy

In [ ]:
# take a single occupation from the adult dataset, and return a single response
def encode_response_sales(response, alpha):
    if random.random() < (1/2 + alpha):
        if response == 'Sales':
            return 1
        else:
            return 0
    else:
        if response == 'Sales':
            return 0
        else:
            return 1
    
def decode_responses_sales(responses, alpha):
    return sum([((y - 1/2 + alpha) / 2 * alpha) for y in responses])

alpha = 0.05
responses = [encode_response_sales(r, alpha) for r in adult['Occupation']]
decode_responses_sales(responses, alpha)

#### Version from the [book](https://programming-dp.com/ch13.html), mechanism by Dwork and Roth

In [ ]:
def encode_rand_resp_yes_no(true_response):
    if random.random() < (1/2 + alpha):
        return true_response
    else:
        if random.random() < (1/2 + alpha):
            return True
        else:
            return False

- With probability 1/2 - alpha each respondant responds randomly and with probability 1/2 + 0.05, each random response is a "yes", so the probability that a respondant responds “yes” by random cis (1/2 - 0.05) * (1/2 + 0.05) = 0.2475.
- The other factor we need to consider is that half of the respondants answer randomly, but some of the random respondants might actually be salespeople. But, since we split the respondants into “truth” and “random” groups randomly, we can hope that there are roughly the same number of salespeople in both groups. Since alpha is quite small.

In [ ]:
def decode_rand_resp_yes_no(responses):
    # Decode the results of randomized responses
    all_yes = np.sum(responses)
    
    # Subtract the number of fake yesses
    fake_yes = (0.2475)*len(responses)
    true_yes = all_yes - fake_yes
    
    # Multiply by 2
    return 2*true_yes

In [ ]:
# take a single occupation from the adult dataset, and return a single response
def encode_response_sales(response):
    randomized_response = encode_rand_resp_yes_no(response == 'Sales')
    return randomized_response
    
def decode_responses_sales(responses):
    approximate_yesses = decode_rand_resp_yes_no(responses)
    return approximate_yesses

responses = [encode_response_sales(r) for r in adult['Occupation']]
decode_responses_sales(responses)

Better answer.

In [ ]:
# How accurate is the answer above?
true_sales = np.sum(adult['Occupation'] == 'Sales')
print('True number of salespeople:', true_sales)

We have that $$\epsilon = \ln (\frac{\alpha+1}{\alpha-1}) = 0.1$$